In [ ]:
# When using a colab notebook:
#!git clone https://github.com/Aleph-Alpha/examples.git
#!pip install -r examples/requirements.txt
#!cp examples/bootcamp/data.md data.md

In [1]:
from aleph_alpha_client import Client, Prompt, CompletionRequest,ExplanationRequest, TextControl, EvaluationRequest, TargetGranularity
from scipy import spatial
import numpy as np
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

client = Client(token=os.getenv("AA_TOKEN"))

#### Lets learn about Attention Manipulation

In [10]:
text = "The quick brown fox jumps over the lazy dog.\nThe color of the fox is"
# Here we define a TextControl that will be used to control the attention on the prompt.
# Change the factor to 0.0 to see what happens.
control = TextControl(start=10, length=5, factor=1)
prompt = Prompt.from_text(text, controls=[control])

request = CompletionRequest(prompt=prompt, maximum_tokens=10, stop_sequences=["."])
result = client.complete(request = request, model="luminous-extended")
print(result.completions[0].completion)

 brown


We see, that changing the attention changes the output of the model. Let's see how that can help us in different scenarios.

In [21]:
# Let's try that again. but this time, we want to get traceable explanations.
text = "The quick brown fox jumps over the lazy dog.\nThe color of the fox is"

controls = []
# create a control for each word in the prompt
for i, word in enumerate(text.split()):
    # we want to control the attention on each word
    # so we set the factor to 1.0
    #get the starting index of the word
    start = text.find(word)
    control = TextControl(start=start, length=len(word), factor=0.1)
    controls.append(control)
    
eval_scores = []
for control in controls:
    prompt = Prompt.from_text(text, controls=[control])
    
    request = EvaluationRequest(prompt=prompt, completion_expected=" brown")
    score = client.evaluate(request = request, model="luminous-extended")
    print(f"The control of '{text[control.start:control.start+control.length]}' is: {score.result['log_perplexity']}")
    


The control of 'The' is: 0.4304359
The control of 'quick' is: 0.8072545
The control of 'brown' is: 1.9476956
The control of 'fox' is: 0.29378745
The control of 'jumps' is: 0.8143241
The control of 'over' is: 0.684362
The control of 'the' is: 0.88111794
The control of 'lazy' is: 0.51506716
The control of 'dog.' is: 1.3384898
The control of 'The' is: 0.4304359
The control of 'color' is: 1.4661577
The control of 'of' is: 2.0097778
The control of 'the' is: 0.88111794
The control of 'fox' is: 0.29378745
The control of 'is' is: 0.7834949


### Let's use an explantation request to find out what the model is looking at

In [13]:
exp_req = ExplanationRequest(Prompt.from_text(text), " brown", control_factor=0.3)
response_explain = client.explain(exp_req, model="luminous-extended")

explanations = response_explain[1][0].items[0][0]

for item in explanations:
    start = item.start
    end = item.start + item.length
    print(f"""EXPLAINED TEXT: {text[start:end]}
Score: {np.round(item.score, decimals=3)}""")

EXPLAINED TEXT: The
Score: -0.261
EXPLAINED TEXT: quick
Score: 0.077
EXPLAINED TEXT: brown
Score: 1.354
EXPLAINED TEXT: fox
Score: -0.41
EXPLAINED TEXT: jumps
Score: 0.13
EXPLAINED TEXT: over
Score: 0.06
EXPLAINED TEXT: the
Score: 0.142
EXPLAINED TEXT: lazy
Score: -0.182
EXPLAINED TEXT: dog
Score: -0.188
EXPLAINED TEXT: .
Score: 0.786
EXPLAINED TEXT: 

Score: 0.063
EXPLAINED TEXT: The
Score: 0.036
EXPLAINED TEXT: color
Score: 0.775
EXPLAINED TEXT: of
Score: 1.274
EXPLAINED TEXT: the
Score: 0.779
EXPLAINED TEXT: fox
Score: 2.327
EXPLAINED TEXT: is
Score: 0.11


In [16]:
# Read the data in the data.md file
with open("data.md", "r") as f:
    data = f.read()
    
# Split the data into a list of texts
texts = data.split("#")[1:]

print(f"data: {data[:100]}")

data: # Taking control of the digital transformation
The digital transformation is in full swing. The tech


In [17]:
answers_prompt = f"""### Instructions: Solve the task based on the text below".

### Input:
{texts[4]}

### Task: When was the declaration signed?

### Reponse:"""

response = client.complete(CompletionRequest(prompt=Prompt.from_text(answers_prompt), maximum_tokens=100, stop_sequences=["###"]), model="luminous-base-control")
answer = response.completions[0].completion

exp_req = ExplanationRequest(Prompt.from_text(answers_prompt), answer, control_factor=0.1, prompt_granularity="paragraph", target_granularity=TargetGranularity.Complete)
response_explain = client.explain(exp_req, model="luminous-extended-control")

explanations = response_explain[1][0].items[0][0]

for item in explanations:
    start = item.start
    end = item.start + item.length
    print(f"""EXPLAINED TEXT: {answers_prompt[start:end]}
Score: {np.round(item.score, decimals=3)}""")

EXPLAINED TEXT: ### Instructions: Solve the task based on the text below".
Score: 0.391
EXPLAINED TEXT: ### Input:
 Breakthrough in the ambitious EU expansion of renewable energy
EU Council adopts new Directive for the use of renewable energies in the Member States
Score: -0.036
EXPLAINED TEXT: On 16 June, the Member States of the European Union approved a comprehensive revision of the EU Renewable Energy Directive (RED) in the Permanent Representatives Committee. The European target for renewable energies in 2030 is raised significantly from 32% to 45%. This means a doubling of the share of renewable energies compared to the current level of just under 22% in 2021.
Score: 0.176
EXPLAINED TEXT: This gives a great boost to the expansion of renewables: the planned expansion of renewable energies by 2030 will thus be roughly doubled. To achieve these new targets, the EU will have to install more than 100 GW of new wind turbines and solar plants every year. For Germany, this means that the

In [22]:
exp_req = ExplanationRequest(Prompt.from_text(answers_prompt), " The USA have social elements.", control_factor=0.1, prompt_granularity="paragraph", target_granularity=TargetGranularity.Complete)
response_explain = client.explain(exp_req, model="luminous-extended")

explanations = response_explain[1][0].items[0][0]

for item in explanations:
    start = item.start
    end = item.start + item.length
    print(f"""EXPLAINED TEXT: {answers_prompt[start:end]}
Score: {np.round(item.score, decimals=3)}""")

EXPLAINED TEXT: ### Instructions: Solve the task based on the text below".
Score: -0.761
EXPLAINED TEXT: ### Input:
 Breakthrough in the ambitious EU expansion of renewable energy
EU Council adopts new Directive for the use of renewable energies in the Member States
Score: -1.047
EXPLAINED TEXT: On 16 June, the Member States of the European Union approved a comprehensive revision of the EU Renewable Energy Directive (RED) in the Permanent Representatives Committee. The European target for renewable energies in 2030 is raised significantly from 32% to 45%. This means a doubling of the share of renewable energies compared to the current level of just under 22% in 2021.
Score: -0.631
EXPLAINED TEXT: This gives a great boost to the expansion of renewables: the planned expansion of renewable energies by 2030 will thus be roughly doubled. To achieve these new targets, the EU will have to install more than 100 GW of new wind turbines and solar plants every year. For Germany, this means that t